An Analysis of Train Derailments

Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

# import data
full_data = pd.read_csv("Rail_Equipment_Accident_Incident_Data.csv")
full_data.head()

C:\Users\Gian\AppData\Local\Temp\ipykernel_25964\3595201295.py:7: DtypeWarning: Columns (26,27,43,46,47,48,50,59,61,62,63,64,65,66,67,68,69,71,129,130,131,136) have mixed types. Specify dtype option on import or set low_memory=False.
  full_data = pd.read_csv("Rail_Equipment_Accident_Incident_Data.csv")


,Reporting Railroad Code,Reporting Railroad Name,Report Year,Accident Number,PDF Link,Accident Year,Accident Month,Other Railroad Code,Other Railroad Name,Other Accident Number,...,Other Railroad SMT Grouping,Other Parent Railroad Company Code,Other Parent Railroad Company Name,Other Railroad Holding Company,Maintenance Railroad Company Grouping,Maintenance Railroad Class,Maintenance Railroad SMT Grouping,Maintenance Parent Railroad Company Code,Maintenance Parent Railroad Company Name,Maintenance Railroad Holding Company
0,NICD,Northern Indiana Commuter Transportation District,2017.0,N1700013,https://safetydata.fra.dot.gov/Officeofsafety/...,17.0,6.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Passenger (Formerly Commuter),Class 3,SMT-4 - CP/CN/CCD,NIRC,NORTHEAST ILLINOIS REGIONAL COMMUTER RAIL CORP...,Not Assigned
1,NIRC,NORTHEAST ILLINOIS REGIONAL COMMUTER RAIL CORP...,2017.0,201707188,https://safetydata.fra.dot.gov/Officeofsafety/...,17.0,6.0,NICD,Northern Indiana Commuter Transportation District,N1700013,...,SMT-4 - CP/CN/CCD,NICD,Northern Indiana Commuter Transportation District,Not Assigned,Passenger (Formerly Commuter),Class 3,SMT-4 - CP/CN/CCD,NIRC,NORTHEAST ILLINOIS REGIONAL COMMUTER RAIL CORP...,Not Assigned
2,CR,Conrail,1981.0,0420001,https://safetydata.fra.dot.gov/Officeofsafety/...,81.0,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Class 3,NaN,CRSH,Consolidated Rail Corporation,Not Assigned
3,IC,Illinois Central Railroad Company,2007.0,543854,https://safetydata.fra.dot.gov/Officeofsafety/...,7.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Shippers,Class 3,NaN,CAEZ,Cargill Elevator,Not Assigned
4,IC,Illinois Central Railroad Company,2017.0,938223,https://safetydata.fra.dot.gov/Officeofsafety/...,17.0,10.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Class 1,SMT-4 - CP/CN/CCD,CN,Canadian National - North America,Canadian National - North America


# Data Cleaning

In [2]:
features_to_drop = ["Reporting Railroad Code",
                 "PDF Link",
                 "Other Railroad Code",
                 "Maintenance Railroad Code",
                 "Accident Type Code",
                 "State Code",
                 "County Code",
                 "Visibility Code",
                 "Weather Condition Code",
                 "Track Type Code",
                 "Train Direction Code",
                 "Equipment Type Code",
                 "Signalization Code",
                 "Method of Operation Code",
                 "Remote Control Locomotive Code",
                 "Primary Accident Cause Code",
                 "Contributing Accident Cause Code",
                 "Accident Cause Code",
                 "Class Code",
                 "Reporting Parent Railroad Company Code",
                 "Other Parent Railroad Company Code",
                 "Maintenance Parent Railroad Company Code"
]

data = full_data.drop(features_to_drop,axis=1)
data.head()

,Reporting Railroad Name,Report Year,Accident Number,Accident Year,Accident Month,Other Railroad Name,Other Accident Number,Other Accident Year,Other Accident Month,Maintenance Railroad Name,...,Other Railroad Company Grouping,Other Railroad Class,Other Railroad SMT Grouping,Other Parent Railroad Company Name,Other Railroad Holding Company,Maintenance Railroad Company Grouping,Maintenance Railroad Class,Maintenance Railroad SMT Grouping,Maintenance Parent Railroad Company Name,Maintenance Railroad Holding Company
0,Northern Indiana Commuter Transportation District,2017.0,N1700013,17.0,6.0,NaN,NaN,NaN,NaN,NORTHEAST ILLINOIS REGIONAL COMMUTER RAIL CORP...,...,NaN,NaN,NaN,NaN,NaN,Passenger (Formerly Commuter),Class 3,SMT-4 - CP/CN/CCD,NORTHEAST ILLINOIS REGIONAL COMMUTER RAIL CORP...,Not Assigned
1,NORTHEAST ILLINOIS REGIONAL COMMUTER RAIL CORP...,2017.0,201707188,17.0,6.0,Northern Indiana Commuter Transportation District,N1700013,17.0,6.0,NORTHEAST ILLINOIS REGIONAL COMMUTER RAIL CORP...,...,Passenger (Formerly Commuter),Class 3,SMT-4 - CP/CN/CCD,Northern Indiana Commuter Transportation District,Not Assigned,Passenger (Formerly Commuter),Class 3,SMT-4 - CP/CN/CCD,NORTHEAST ILLINOIS REGIONAL COMMUTER RAIL CORP...,Not Assigned
2,Conrail,1981.0,0420001,81.0,4.0,NaN,NaN,NaN,NaN,Conrail,...,NaN,NaN,NaN,NaN,NaN,NaN,Class 3,NaN,Consolidated Rail Corporation,Not Assigned
3,Illinois Central Railroad Company,2007.0,543854,7.0,1.0,NaN,NaN,NaN,NaN,Cargill Elevator,...,NaN,NaN,NaN,NaN,NaN,Shippers,Class 3,NaN,Cargill Elevator,Not Assigned
4,Illinois Central Railroad Company,2017.0,938223,17.0,10.0,NaN,NaN,NaN,NaN,Illinois Central Railroad Company,...,NaN,NaN,NaN,NaN,NaN,NaN,Class 1,SMT-4 - CP/CN/CCD,Canadian National - North America,Canadian National - North America


# Regularization

# Saving

In [3]:
# save data to file
data.to_csv("cleaned_data.csv",index=False) # index=False to not save the index column

In [4]:
# saves all column names to separate file in order to find different variables
with open("column_names.txt", "w") as f:
    for s in data.columns:
        f.write(s + "\n")
